In [1]:
import findspark

findspark.init()

from pyspark import SparkContext
import numpy as np


sc = SparkContext("local[*]", "Media practica")
rdd = sc.textFile("/home/ciruu/CLionProjects/practica2SPAI/test.csv")

24/10/04 20:08:13 WARN Utils: Your hostname, Ciruu resolves to a loopback address: 127.0.1.1; using 10.150.83.214 instead (on interface wlp2s0)
24/10/04 20:08:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/04 20:08:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/04 20:08:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
def readRow(row):
    arr = row.split(",")
    lst = np.array(arr).astype(np.float64)
    return np.array([float(x) for x in lst[:-1]]), int(lst[-1])

def suma(el1, el2):
    return el1 + el2

In [3]:
rdd1 = rdd.map(readRow)
print(rdd1.take(2))

num = rdd.count()
print(num)

# Dividimos entre el número total de filas
def divide_by_num(tup):
    tup[0] = tup[0] / num
    return tup


rdd2 = rdd1.reduce(lambda x, y: (x[0] + y[0], x[1]))

print(rdd2)
medias = rdd2[0] / num
# Calculamos la media por cada columna con índice I
#medias = rdd3.collect()
print(f"MEDIAS: {medias}")
#print(medias)

[(array([ 3.00388642e+00,  2.07699955e+03, -4.41037030e-05,  1.00000002e+01,
        1.29114942e+04,  6.99998079e+01,  1.20000000e+01, -2.35688188e-08,
        1.99000002e+02,  2.46836961e+09,  2.46836839e+09]), 1), (array([ 9.50811331e+02,  4.12285694e+04,  5.30000498e+01,  9.99999745e-01,
        1.38579999e+04,  4.29592488e+06,  1.30000000e+01, -2.35688188e-08,
        8.00000498e+00,  2.46836962e+09,  2.46837255e+09]), 1)]
5000
(array([6.15926485e+06, 1.03681121e+08, 3.63034537e+07, 3.00722501e+08,
       6.24743675e+11, 7.62920618e+10, 4.61115064e+04, 9.28249311e+03,
       6.20960335e+05, 1.06658485e+13, 1.13517042e+13]), 1)
MEDIAS: [1.23185297e+03 2.07362242e+04 7.26069074e+03 6.01445003e+04
 1.24948735e+08 1.52584124e+07 9.22230129e+00 1.85649862e+00
 1.24192067e+02 2.13316969e+09 2.27034084e+09]


In [11]:
from math import sqrt
# Función para obtener la media de un índice en concreto en un array
def get_media_from_tup_array(arr, index):
    for e in arr:
        if e[0] == index:
            return e[1]
        
        
# Función para calcular la varianza a partir de una tupla
def calc_var(tup):
    #mean = get_media_from_tup_array(medias, tup[0])
    return ((tup[0] - medias) ** 2), tup[1]

In [12]:
# Caćulo de la varianza a partir de las tuplas
rdd4 = rdd1.map(calc_var)
print(rdd4.take(2))
rdd5 = rdd4.reduce(lambda x, y: (x[0] + y[0], x[1]))



print(f"VARIANZA: {rdd5[0]/num}")

[(array([1.51007007e+06, 3.48166664e+08, 5.27176306e+07, 3.61615812e+09,
       1.56089600e+16, 2.32817011e+14, 7.71561013e+00, 3.44658722e+00,
       5.59622713e+03, 1.12358987e+17, 3.92149115e+16]), 1), (array([7.89844029e+04, 4.19936211e+08, 5.19508051e+07, 3.61724063e+09,
       1.56087235e+16, 1.20176132e+14, 1.42710073e+01, 3.44658722e+00,
       1.35005953e+04, 1.12358990e+17, 3.92165577e+16]), 1)]
VARIANZA: [2.51781240e+06 5.70220913e+08 2.81562405e+08 1.71636513e+10
 5.62320496e+16 1.89501185e+15 2.73272537e+01 4.39041738e+00
 8.19803933e+03 5.10634476e+17 1.67149262e+18]


In [18]:
#print(rdd5)
rddx = rdd1.map(lambda x: (x[0])).stdev()
print(rddx)
print(f"DESVIACIÓN ESTÁNDAR: {np.sqrt(rdd5[0]/num)}")

[1.58676161e+03 2.38792988e+04 1.67798214e+04 1.31010119e+05
 2.37132979e+08 4.35317339e+07 5.22754758e+00 2.09533228e+00
 9.05430247e+01 7.14586927e+08 1.29286218e+09]
DESVIACIÓN ESTÁNDAR: [1.58676161e+03 2.38792988e+04 1.67798214e+04 1.31010119e+05
 2.37132979e+08 4.35317339e+07 5.22754758e+00 2.09533228e+00
 9.05430247e+01 7.14586927e+08 1.29286218e+09]


# PARTE 2 NORMALIZACIÓN

In [28]:
array_medias = medias
array_stdev = np.sqrt(rdd5[0]/num)
print(array_stdev)
print(array_medias)

[1.58676161e+03 2.38792988e+04 1.67798214e+04 1.31010119e+05
 2.37132979e+08 4.35317339e+07 5.22754758e+00 2.09533228e+00
 9.05430247e+01 7.14586927e+08 1.29286218e+09]
[1.23185297e+03 2.07362242e+04 7.26069074e+03 6.01445003e+04
 1.24948735e+08 1.52584124e+07 9.22230129e+00 1.85649862e+00
 1.24192067e+02 2.13316969e+09 2.27034084e+09]


In [29]:
def normalize(tup):
    data = tup[0]
    normalized_data = []
    i = 0
    for e in data:
        normalized_data.append((e - array_medias[i]) / array_stdev[i])  
        i+=1
    return np.array(normalized_data), tup[1]

In [31]:
rdd2_1 = rdd1.map(normalize)
#med = rdd2_1.map(lambda x: x[0]).mean()
#print(med)
#print(rdd2_1.collect())
stdevv = rdd2_1.map(lambda x: x[0]).stdev()
print(stdevv)
#print(rdd1.collect())
#print(rdd2_1.collect())

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
